<font color='lightblue'>Nom Etudiant 1: ELLIETHY Ossama 28712429
<br>
Nom Etudiant 2: HORCHANI Samy 28706765 </font>

# TME 3.2: Projet Detection de motifs


## Recheche de pattern (motifs) en utilisant les algoritmes randomisés

Les algorithmes randomisés prendre des décisions aléatoire plutôt que déterministes.
l'algorithme s'exécute différemment à chaque fois. Ils sont couramment utilisés dans situations où aucun algorithme exact et rapide est connu. Nous allons d'abord implémenter l'algorithm random Projections.


1\.  Nous allons réutiliser les fonctions du precedent pour générer t séquences artificielles de taille n, et implanter dans chaque séquence un motif de taille k à des positions aléatoires avec v substitutions choisies aléatoirement. Nous allons faire varier le motifs dans 50% de cas.

In [1]:
import random
import numpy as np

nuc = ('A', 'C', 'G', 'T')

k=8 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence
f= 0.5

In [2]:
from math import floor

def insertMotif(sequence, motif, position):
    return sequence[:position] + motif + sequence[position:]

def generateRandomSequence(n, upper=True):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront en majuscule, False minuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    sequence = ""
    
    for i in range(n):
        s = random.choice(nuc)

        if(upper):
            sequence = sequence + s.upper()
        else:
            sequence = sequence + s.lower()

    return sequence


def modifierMotif(motif, nbpos,  upper=True):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront majuscule, False minuscule
    sortie motifM: motif modifié
    """
    motifM = list(motif)
    pos_modifie = []

    
    for i in range(nbpos):
        random_pos = random.randint(0,len(motifM)-1)

        while(random_pos in pos_modifie):
            random_pos = random.randint(0,len(motifM)-1)

        pos_modifie.append(random_pos) #sauvegarde de la position pour pas la modifier 2 fois
        random_nuc = random.choice(nuc)

        while(random_nuc == motif[random_pos]): #pour pas implanter le meme motif 
            random_nuc = random.choice(nuc)

        if(upper):
            motifM[random_pos] = random_nuc.upper()
        else:
            motifM[random_pos] = random_nuc.lower()
        
    return "".join(motifM)

#tester modifMotif
print (modifierMotif("acg", 2))


def implantMotifVar(k, v, t, n, f):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    entrée f : frequence de variation du motif.
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus motif doit être égal à t, pensez à générer de séquence aléatoire de taille t-k pour pouvoir implanter un motif de taille k
    """
    motif = generateRandomSequence(k, False)
    sequences = []
    
    for i in range(t):

        seq = generateRandomSequence(n-k)
        motif = generateRandomSequence(k, False) #pour mieux visualiser les motifs implantés on les inserts en minuscule
        #print("seq = " + seq)
        #print(motif)
        if random.uniform(0.0, 1.0) < f:
            motif = modifierMotif(motif, v)
        #print(motif)
        
        sequences.append(insertMotif(seq, motif, random.randint(0, len(seq)-1)))
        
    return sequences

adn = implantMotifVar(k, v, t, n, f)
print (adn)

adn  = [s.upper() for s in adn]
print (adn)

CcC
['aggtcaaaTG', 'cGagaacgTA', 'aattggTaGG']
['AGGTCAAATG', 'CGAGAACGTA', 'AATTGGTAGG']


2\. Nous allons implémenter l'algorithme ``randomProjection``. D'abord, faites la fonction `getRandomFixePositions` pour générer une projection de p à k, voir un exemple dans les slides de cours. Faire aussi la fonction `generateKey` qui extrait les caractères du motif puis génère une cle qui représente la projection.

In [3]:
def getRandomFixePositions(p, k):
    """
    Genere une projection de p vers k
    entrée p: nombre de positions choisi pour la projection 
    entrée k: nombre de positions du motif original
    sortie projection: liste de positions choisi aléatoirement
    """
    projection = []
    #count = 0
    
    for e in range(p):
        p= random.randint(0, k-1)
        #Pour ne pas avoir à 2 reprises un meme indice
        if p in projection: #On doit tirer un nouvel indice aléatoirevement
            projConnue = True
            while projConnue:
                p= random.randint(0, k-1)
                if p not in projection:
                    projConnue = False
        
        #Ajout de la projection à la liste
        projection.append(p)
    
    return projection

lR = getRandomFixePositions(4, 7)
lR.sort()
print (lR)

def generateKey(projection, motif):
    """
    extrait les caractères du motif et génère la cle de la projection
    entrée projection : liste de positions qui represent la projection
    entrée motif : motif de taille k
    sortie cle : cle de la projection
    """
    cle = ""
    
    for i in (projection):
        if i< len(motif):
            cle= cle + motif[i]
    
    return cle

key=generateKey(lR, "01234567")
print(key)

[1, 2, 3, 6]
1236


3\. Implémenter l'algorithme ``randomProjection``. Bonnus : Pour ameliorer la performance vous pouvez abandonner les motifs de taille k peu complexes.

In [4]:
def islowComplexe(motif, nb):
    """
    Renvoie un boolean pr dire si un motif est peu complex 
    entrée motifs: motif
    sortie motifsClean: boolean
    """
    
    SetDesCaractèresComposantsLeMotif = set(motif) #Suppression des doublons du motifs afin d'obtenir un set avec que les différents caractères qui le compose
    lenSet = len(SetDesCaractèresComposantsLeMotif)
    compteur=0
    
    if lenSet > 2:#si le motif k admet moins de 3 caractères différents il est ignorer car focément 2 meme lettres répétées ou la meme lettre est répétée plusieur fois à la suite
        #Test pour savoir si un mm caractère du motif n'est pas présent plus de nb fois à la suite
        for car in SetDesCaractèresComposantsLeMotif :
            mot=''
            for i in range (nb):
                mot=mot+car

            for y in range (len(motif)-nb+1):
                tmp=motif[y:y+nb]
                if mot==tmp:
                    return True
            
            compteur=compteur+1
            if compteur == lenSet:
                return False         
            
    return True

#motif = 'AATAATC'
#res= islowComplexe(motif, 4)
#print(res)

def randomProjection(k, p, sequences):
    """
    Implémente l'algorithme randomProjection
    entrée k : taille du motif
    entrée p : nombre de positions de la projection 
    entrée sequences : matrice de dimension txn contenant les séquences 
    sortie motifs : dictionaire, cle = projection, valeur= frequence
    sortie motifsSeq:  dictionaire, cle = projection, valeur= original motif
    """
    motifs  = {}; motifsSeq  = {}
    
    p=getRandomFixePositions(p, k)
    p.sort()
    #print(p)
    
    for s in sequences:
        if len(s)>=k:
            for i in range(len(s)-k+1):
                motif = s[i:i+k]
                #Dés que je tire un motif on met un boolean à vrai ou faux en fct de si le motfi est peu comple ou pas 
                isLow=islowComplexe(motif, 5)
                
                #Ensuite si il est peu complex on l'insere pas dans les bucket dans le dico
                if not(isLow):
                    
                    key=generateKey(p, motif)
                    if key in motifs.keys() and key in motifsSeq.keys():
                        #Si la clé existe déja
                            motifs[key]+=1
                            motifsSeq[key]=motifsSeq[key]+[motif]
                        
                    else:
                        #Si la clé n'existe pas
                        motifs[key]=1
                        motifsSeq[key]=[motif]
    
    
    return motifs, motifsSeq

#motifsSort = sorted(motifs, reverse=True, key=motifs.get)

adnTest = ['TTAACGGCAC', 'GCTCACGCCA', 'TACCGGCCGT']
motifsProj, motifsSeq = randomProjection(7, 4, adnTest)
print (motifsProj)
print (motifsSeq)

#motifsProj => {'TACG': 1, 'TCGC': 3, 'AGGA': 1, 'AGCC': 1, 'GCAG': 1, 'CACC': 1, 'CGCA': 1, 'AGGC': 1, 'CGCG': 1, 'CCCT': 1}
#motifsSeq => {'TACG': ['TTAACGG'], 'TCGC': ['TAACGGC', 'TCACGCC', 'TACCGGC'], 'AGGA': ['AACGGCA'], 'AGCC': ['ACGGCAC'], 'GCAG': ['GCTCACG'], 'CACC': ['CTCACGC'], 'CGCA': ['CACGCCA'], 'AGGC': ['ACCGGCC'], 'CGCG': ['CCGGCCG'], 'CCCT': ['CGGCCGT']}

{'TACG': 1, 'AAGG': 1, 'ACGC': 1, 'CGCA': 1, 'CTAC': 1, 'TCCG': 1, 'CAGC': 1, 'ACCC': 1, 'ACGG': 1, 'CCGC': 1, 'GGCG': 1}
{'TACG': ['TTAACGG'], 'AAGG': ['TAACGGC'], 'ACGC': ['AACGGCA'], 'CGCA': ['ACGGCAC'], 'CTAC': ['GCTCACG'], 'TCCG': ['CTCACGC'], 'CAGC': ['TCACGCC'], 'ACCC': ['CACGCCA'], 'ACGG': ['TACCGGC'], 'CCGC': ['ACCGGCC'], 'GGCG': ['CGGCCGT']}


4\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour augmenter les chances de le trouver. 

reponse: 

Le motif implémenté a pour clé fixe TCGC associée à la projection car c'est la bucket la plus grande. Trois motifs ont été identifiés dans l'exemple, mais avec des variations pour des positions non projetées.

Étant donné que les projections sont aléatoires, la clé TCGC peut être obtenue, mais pas avec les bonnes projections aléatoires pour retrouver le motif avec des variations. Par conséquent, l'algorithme randomProjection doit être exécuté un grand nombre de fois pour obtenir un résultat satisfaisant.

D'autres projections permettent tout de même de tirer des conclusions. Par exemple, la bucket la plus grande avec la projection [0, 2, 3, 6] permet d'observer une clé unique ayant la plus grande fréquence, c'est-à-dire 'TACC': 2, ainsi que 'TACC': ['TAACGGC', 'TCACGCC']. Cela donne une idée de l'aspect possible du motif, d'autant plus que ce phénomène se répète pour de nombreuses projections où une unique liste (de taille 2) est plus longue que toutes les autres qui ont une taille de 1.

5\. Implémenter la version itérative de l’algorithme ``randomProjection``. 

In [5]:

#Construire matrice de fréquence
def profile(motifs, k, nuc):
    """
    Construire une matrice de fréquence de dimension k x |nuc|
    entrée motifs: liste de motifs
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie MF: matrice de fréquence
    """
    nDict = {nuc[i]: i for i in range(len(nuc))}
    q = len(nuc)
    PWM = np.ones((q, k))
    for col in range(k):
        for line in range(len(motifs)):
            # try:
            nucleotide = motifs[line][col]
            # except IndexError:
                # print(line, col, motifs)
            PWM[nDict[nucleotide]][col] += 1
    return PWM

def getScore(MF, k):
    """
    Renvoie le score de MF, la somme des max de chaque colonne
    entrée MF: matrice de fréquence
    entrée k: taille du motif
    sortie sc: score
    """
    mf = np.mat(MF)
    maxArray = np.max(mf, axis=0)
    return np.sum(maxArray)


In [6]:
def randomProjIt(sequences, k, v, nuc, It):
    """
    Implémente l'algorithme randomProjection version iteractive
    entrée sequences : matrice de dimension txn contenant les séquences 
    entrée k : nombre de positions du motif
    entrée v : nombre de positions de la projection 
    entrée nuc : alphabet
    entrée It: nombre d'iterations
    sortie score : meilleur score
    sortie motifs :  liste de motifs associés au meilleur score
    """
    motifss = []; scores = 0
    
    
    for i in range(It):
        (motifs, motifsSeq)=randomProjection(k, v, sequences)
        keys= motifsSeq.keys()
        
        for ke in keys:
            MF=profile(motifsSeq[ke], k, nuc)
            scoretmp=getScore(MF, k)
            
            if scoretmp> scores:
                scores= scoretmp
                motifss= motifsSeq[ke]  
                
    return scores, motifss

score, seqsMotif = randomProjIt(adnTest, 7, 4, nuc, 100)

print (score, seqsMotif)


25.0 ['TAACGGC', 'TCACGCC', 'TACCGGC']


6\. Tester l'algorithme  ``randomProjection`` sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats. Puis générér le LOGO du motif trouvé.

In [7]:
def reverseComplement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in reversed(seq)])

def complement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in seq])

def printTopFMotifsFreq(motifs, m, rev=False):
    motifsRet = {}
    motifsSort = sorted(motifs, reverse=True, key=motifs.get)
    i = 0
    while (i < m):
        motifPrint = motifsSort[i]
        print (motifsSort[i])
        if rev:
            motifPrint = reverseComplement(motifsSort[i])
        print (i, motifPrint, "-", motifs[motifsSort[i]])
        i = i + 1
        
printTopFMotifsFreq(motifsProj, 10)
#print (motifsSeq['TAGG'])

TACG
0 TACG - 1
AAGG
1 AAGG - 1
ACGC
2 ACGC - 1
CGCA
3 CGCA - 1
CTAC
4 CTAC - 1
TCCG
5 TCCG - 1
CAGC
6 CAGC - 1
ACCC
7 ACCC - 1
ACGG
8 ACGG - 1
CCGC
9 CCGC - 1


In [9]:
k=7; p=4; n=80

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    #sequence = [sequenceStr]
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    sequenceRet = [x for x in sequence if x]
    return sequenceRet

genome = "Sequence_by_Peaks_1.fasta"

sequencesChip   = readFasta(genome, n)
t = len(sequencesChip)
print (sequencesChip[8], t, n, k)
revSequences = [reverseComplement(m) for m in sequencesChip]

sequences = sequencesChip + revSequences

score, seqsMotif = randomProjIt(sequences, k, p, nuc, 100)

print(score, seqsMotif)


CATTTTGATCCATTGTGTGTTAAAGCACAGTGGGACCCAGGAATAGTAGAGAGAAGGAAAGTGGGGGAAGGGTAGCAGTG 399 80 7
2356.0 ['TTTCAAA', 'CTTCCAA', 'GTTAAAA', 'CTTCAAA', 'ATTTCAA', 'GTTGTAA', 'GTTAAAA', 'ATTTCAA', 'ATTCAAA', 'ATTAGAA', 'TTTCTAA', 'TTTGCAA', 'TTTGCAA', 'GTTGTAA', 'CTTGAAA', 'ATTACAA', 'CTTTCAA', 'TTTCAAA', 'TTTAGAA', 'TTTACAA', 'CTTAAAA', 'TTTTGAA', 'GTTTGAA', 'TTTGAAA', 'GTTTCAA', 'ATTAGAA', 'GTTCTAA', 'GTTCCAA', 'CTTACAA', 'ATTACAA', 'CTTGCAA', 'GTTTTAA', 'CTTGAAA', 'ATTGCAA', 'CTTGCAA', 'ATTGCAA', 'CTTCTAA', 'ATTCGAA', 'GTTGTAA', 'TTTACAA', 'CTTAAAA', 'CTTAAAA', 'TTTCCAA', 'TTTAGAA', 'CTTAGAA', 'GTTTAAA', 'GTTAAAA', 'GTTGCAA', 'CTTATAA', 'GTTACAA', 'CTTTGAA', 'ATTCTAA', 'TTTGCAA', 'TTTGTAA', 'GTTCAAA', 'ATTTCAA', 'TTTGTAA', 'ATTCAAA', 'ATTAGAA', 'GTTTTAA', 'GTTAGAA', 'ATTGAAA', 'ATTTGAA', 'ATTAGAA', 'CTTAGAA', 'TTTCCAA', 'GTTATAA', 'ATTTGAA', 'CTTGAAA', 'CTTCAAA', 'TTTCCAA', 'ATTGTAA', 'TTTTCAA', 'TTTCAAA', 'CTTCCAA', 'ATTTCAA', 'CTTACAA', 'ATTGAAA', 'TTTCGAA', 'CTTGGAA', 'ATTCGAA', 'ATTCAAA', 